In [9]:
pip install xlwt

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install pandas-explode

Note: you may need to restart the kernel to use updated packages.


In [11]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import time
import random
import json
from scholarly import scholarly
import requests
from bs4 import BeautifulSoup
import re
import csv
import pandas as pd

In [12]:
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)

In [13]:
driver = webdriver.Chrome(r"C:\Users\COM01\Downloads\chromedriver_win32 (1)\chromedriver.exe", chrome_options=chrome_options)
driver.get('https://id.elsevier.com/as/authorization.oauth2?platSite=SC%2Fscopus&ui_locales=en-US&scope=openid+profile+email+els_auth_info+els_analytics_info+urn%3Acom%3Aelsevier%3Aidp%3Apolicy%3Aproduct%3Aindv_identity&response_type=code&redirect_uri=https%3A%2F%2Fwww.scopus.com%2Fauthredirect.uri%3FtxGid%3D9a61dba451cc228daa0169ce70312254&state=userLogin%7CtxId%3D55A652A039C8CA86B02E36650AF69927.i-0dcf1575680fbeb33%3A5&authType=SINGLE_SIGN_IN&prompt=login&client_id=SCOPUS')
# log-in by self

C:\Users\COM01\AppData\Local\Temp\ipykernel_3924\1363857600.py:1: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(r"C:\Users\COM01\Downloads\chromedriver_win32 (1)\chromedriver.exe", chrome_options=chrome_options)


In [14]:
# read csv file 
# list scopus id into list_id
data_all = pd.read_csv(r"C:\Users\COM01\Downloads\5065-6330.csv",encoding='latin1')
list_id = ["56667622500", "57222758601"]

In [15]:
scholar = []
for id in list_id:
    driver.get(f'https://www.scopus.com/authid/detail.uri?authorId={id}')
    time.sleep(3)
    
    button = driver.find_element(By.XPATH, '//*[@id="documents-panel"]/div[2]/div/div[2]/div/els-results-layout/div[1]/ul/li[1]/div/div[1]/els-collapsible-panel-v2/div/button')
    button.click()
    
    li_elements = driver.find_elements(By.XPATH, '//*[@id="documents-panel"]/div[2]/div/div[2]/div/els-results-layout/div[1]/ul/li')

    for li in li_elements:
        button = li.find_element(By.XPATH, './/div/div/els-collapsible-panel-v2/div/button')
        button.click()
        

    author_name = driver.find_element(By.XPATH, '//h1[@data-testid="author-profile-name"]').text
    author_id = id 
    num_doc = driver.find_element(By.XPATH, '(//*/span[@data-testid="unclickable-count"])[2]').text
    titile_date = driver.find_element(By.XPATH, '//*[@id="documents-panel"]/div[2]/div/div[2]/div/els-results-layout/div[1]/ul/li[1]/div/div[1]/div[3]/span').text
    
    # show 200 result
    
    try:
        driver.find_element(By.XPATH, '//*[@id="documents-panel"]/div[2]/div/div[2]/div/els-results-layout/els-paginator/nav/els-select/div/label/select/option[5]').click()
        time.sleep(3)
    except:
        pass
    
    research_list = []
    researchs = driver.find_elements(By.XPATH, "//*/els-results-layout//h4")
    for i in range(len(researchs)):
        research_list.append(researchs[i].text)
    
    abs_list = []
    abs = driver.find_elements(By.XPATH, '//*/els-results-layout//els-collapsible-panel-v2/section')
    for i in range(len(abs)):
        abs_list.append(abs[i].text)
    
    print(author_name, "Number of Documents :", num_doc)
    

    scopus = dict()
    scopus['author_id']  = author_id
    scopus['author_name'] = author_name
    scopus['documents_number'] = num_doc
    scopus['date'] = titile_date
    scopus['documents'] = list()
    for i in range(int(num_doc.split(" ")[0])):
        temp_dict = dict()
        temp_dict['title'] = research_list[i]
        
        try : 
            temp_dict['abstraction'] = abs_list[i]
        except :
            abstraction.append('n/a')
            
        scopus['documents'].append(temp_dict)

    scholar.append(scopus)

Norkaew, Saowanee Number of Documents : 9
Chatchawan, Sapphasak Number of Documents : 2


In [16]:
scholar

[{'author_id': '56667622500',
  'author_name': 'Norkaew, Saowanee',
  'documents_number': '9',
  'date': '2022, 12(1), 2946',
  'documents': [{'title': 'Residential environment in relation to self-report of respiratory and asthma symptoms among primary school children in a high-polluted urban area',
    'abstraction': 'Respiratory disease and its complication are the cause of children deaths worldwide every year. Several epidemiological studies pointed out an association between quality of residential in inner city and risk of children health. However, few studies had been focused in high-polluted urban area in low to middle income countries. A cross-sectional study was conducted to investigate the association between residential environments and respiratory symptoms including asthma among 658 primary school children living in urban area of Bangkok Thailand. The International Study of Asthma and Allergies in Childhood (ISAAC) questionnaire was modified to access symptoms during the pas

In [17]:
df1 = pd.DataFrame(scholar, columns = ['author_id', 'author_name', 'documents_number',
                                           'citiations' , 'h-index' , 'co-authors' , 'documents'])

In [18]:
df1

,author_id,author_name,documents_number,citiations,h-index,co-authors,documents
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,[{'title': 'Residential environment in relatio...
1,57222758601,"Chatchawan, Sapphasak",2,NaN,NaN,NaN,[{'title': 'Can sentiment from news headlines ...


In [19]:
df1['documents'][0]

[{'title': 'Residential environment in relation to self-report of respiratory and asthma symptoms among primary school children in a high-polluted urban area',
  'abstraction': 'Respiratory disease and its complication are the cause of children deaths worldwide every year. Several epidemiological studies pointed out an association between quality of residential in inner city and risk of children health. However, few studies had been focused in high-polluted urban area in low to middle income countries. A cross-sectional study was conducted to investigate the association between residential environments and respiratory symptoms including asthma among 658 primary school children living in urban area of Bangkok Thailand. The International Study of Asthma and Allergies in Childhood (ISAAC) questionnaire was modified to access symptoms during the past 12 months. Binary logistic regression model was performed. Living near garment and clothing shop is associated with shortness of breath (AOR 

In [20]:
type(df1['documents'][0])

list

In [21]:
df1['documents'][0][1]

{'title': 'Anthropometric Measurement of Thai Older Farmers for Agricultural Tools and Workplace Design',
 'abstraction': 'Agricultural workers usually perform most occupational operations manually. Mismatch between farmers’ anthropometric dimensions and tools or equipment are known to be contributing factors related discomfort, fatigue, injuries, and biomechanical stress to the users, especially for older farmers. A cross-sectional survey was carried out on 197 male and 284 female older farmers in Nong Suea District, Pathum Thani Province, Thailand. The convenience sampling method was used to select the subjects. Thirty-three anthropometric dimensions were measured. The mean; standard deviations; coefficients of variation; independent t-test; and 5th, 50th, and 95th percentile values were determined. The results revealed differences between dimensions for men and women, indicating that men showed prominent results. Moreover, there was a comparison between some dimensions with the resu

In [22]:
df1=df1.explode('documents')
df1

,author_id,author_name,documents_number,citiations,h-index,co-authors,documents
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'Residential environment in relation...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'Anthropometric Measurement of Thai ...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'Factors Influencing Work Ability am...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'An artificial neural network predic...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'Estimating occupational exposure to...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'Application of anthropometric data ...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,"{'title': 'Knowledge, attitudes, and practices..."
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'An association between organophosph...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'Organophosphate pesticide exposure ...
1,57222758601,"Chatchawan, Sapphasak",2,NaN,NaN,NaN,{'title': 'Can sentiment from news headlines e...


In [23]:
df1['main_title']=df1['documents'].apply(lambda v:v)
df1

,author_id,author_name,documents_number,citiations,h-index,co-authors,documents,main_title
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'Residential environment in relation...,{'title': 'Residential environment in relation...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'Anthropometric Measurement of Thai ...,{'title': 'Anthropometric Measurement of Thai ...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'Factors Influencing Work Ability am...,{'title': 'Factors Influencing Work Ability am...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'An artificial neural network predic...,{'title': 'An artificial neural network predic...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'Estimating occupational exposure to...,{'title': 'Estimating occupational exposure to...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'Application of anthropometric data ...,{'title': 'Application of anthropometric data ...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,"{'title': 'Knowledge, attitudes, and practices...","{'title': 'Knowledge, attitudes, and practices..."
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'An association between organophosph...,{'title': 'An association between organophosph...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'Organophosphate pesticide exposure ...,{'title': 'Organophosphate pesticide exposure ...
1,57222758601,"Chatchawan, Sapphasak",2,NaN,NaN,NaN,{'title': 'Can sentiment from news headlines e...,{'title': 'Can sentiment from news headlines e...


In [24]:
df1['main_abstract']=df1['main_title'].apply(lambda v:v)
df1

,author_id,author_name,documents_number,citiations,h-index,co-authors,documents,main_title,main_abstract
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'Residential environment in relation...,{'title': 'Residential environment in relation...,{'title': 'Residential environment in relation...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'Anthropometric Measurement of Thai ...,{'title': 'Anthropometric Measurement of Thai ...,{'title': 'Anthropometric Measurement of Thai ...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'Factors Influencing Work Ability am...,{'title': 'Factors Influencing Work Ability am...,{'title': 'Factors Influencing Work Ability am...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'An artificial neural network predic...,{'title': 'An artificial neural network predic...,{'title': 'An artificial neural network predic...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'Estimating occupational exposure to...,{'title': 'Estimating occupational exposure to...,{'title': 'Estimating occupational exposure to...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'Application of anthropometric data ...,{'title': 'Application of anthropometric data ...,{'title': 'Application of anthropometric data ...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,"{'title': 'Knowledge, attitudes, and practices...","{'title': 'Knowledge, attitudes, and practices...","{'title': 'Knowledge, attitudes, and practices..."
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'An association between organophosph...,{'title': 'An association between organophosph...,{'title': 'An association between organophosph...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'Organophosphate pesticide exposure ...,{'title': 'Organophosphate pesticide exposure ...,{'title': 'Organophosphate pesticide exposure ...
1,57222758601,"Chatchawan, Sapphasak",2,NaN,NaN,NaN,{'title': 'Can sentiment from news headlines e...,{'title': 'Can sentiment from news headlines e...,{'title': 'Can sentiment from news headlines e...


In [25]:
df1['main_title'].apply(lambda v: v['title'])

0    Residential environment in relation to self-re...
0    Anthropometric Measurement of Thai Older Farme...
0    Factors Influencing Work Ability among the Wor...
0    An artificial neural network prediction model ...
0    Estimating occupational exposure to vocs, svoc...
0    Application of anthropometric data for elderly...
0    Knowledge, attitudes, and practices (KAP) rela...
0    An association between organophosphate pestici...
0    Organophosphate pesticide exposure and dialkyl...
1    Can sentiment from news headlines explain stoc...
1    Do Words in Central Bank Press Releases Affect...
Name: main_title, dtype: object

In [26]:
df1['Title']=df1['main_title'].apply(lambda v:v['title'])
df1

,author_id,author_name,documents_number,citiations,h-index,co-authors,documents,main_title,main_abstract,Title
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'Residential environment in relation...,{'title': 'Residential environment in relation...,{'title': 'Residential environment in relation...,Residential environment in relation to self-re...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'Anthropometric Measurement of Thai ...,{'title': 'Anthropometric Measurement of Thai ...,{'title': 'Anthropometric Measurement of Thai ...,Anthropometric Measurement of Thai Older Farme...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'Factors Influencing Work Ability am...,{'title': 'Factors Influencing Work Ability am...,{'title': 'Factors Influencing Work Ability am...,Factors Influencing Work Ability among the Wor...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'An artificial neural network predic...,{'title': 'An artificial neural network predic...,{'title': 'An artificial neural network predic...,An artificial neural network prediction model ...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'Estimating occupational exposure to...,{'title': 'Estimating occupational exposure to...,{'title': 'Estimating occupational exposure to...,"Estimating occupational exposure to vocs, svoc..."
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'Application of anthropometric data ...,{'title': 'Application of anthropometric data ...,{'title': 'Application of anthropometric data ...,Application of anthropometric data for elderly...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,"{'title': 'Knowledge, attitudes, and practices...","{'title': 'Knowledge, attitudes, and practices...","{'title': 'Knowledge, attitudes, and practices...","Knowledge, attitudes, and practices (KAP) rela..."
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'An association between organophosph...,{'title': 'An association between organophosph...,{'title': 'An association between organophosph...,An association between organophosphate pestici...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'Organophosphate pesticide exposure ...,{'title': 'Organophosphate pesticide exposure ...,{'title': 'Organophosphate pesticide exposure ...,Organophosphate pesticide exposure and dialkyl...
1,57222758601,"Chatchawan, Sapphasak",2,NaN,NaN,NaN,{'title': 'Can sentiment from news headlines e...,{'title': 'Can sentiment from news headlines e...,{'title': 'Can sentiment from news headlines e...,Can sentiment from news headlines explain stoc...


In [27]:
df1['main_abstract'].apply(lambda v: v['abstraction'])

0    Respiratory disease and its complication are t...
0    Agricultural workers usually perform most occu...
0    The ability to work is an essential factor in ...
0    Exposure to indoor air pollutants can cause ad...
0    Thailand is known for its agricultural product...
0    Home environments are the most common injury l...
0    Tobacco is one of the crops using a lot of pes...
0    BACKGROUND: Parkinson's disease (PD) is a ubiq...
0    BACKGROUND: Chlorpyrifos and profenofos are or...
1    The objective of this study was to investigate...
1    The study investigates how financial markets r...
Name: main_abstract, dtype: object

In [28]:
df1['Abstraction']=df1['main_abstract'].apply(lambda v:v['abstraction'])
df1

,author_id,author_name,documents_number,citiations,h-index,co-authors,documents,main_title,main_abstract,Title,Abstraction
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'Residential environment in relation...,{'title': 'Residential environment in relation...,{'title': 'Residential environment in relation...,Residential environment in relation to self-re...,Respiratory disease and its complication are t...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'Anthropometric Measurement of Thai ...,{'title': 'Anthropometric Measurement of Thai ...,{'title': 'Anthropometric Measurement of Thai ...,Anthropometric Measurement of Thai Older Farme...,Agricultural workers usually perform most occu...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'Factors Influencing Work Ability am...,{'title': 'Factors Influencing Work Ability am...,{'title': 'Factors Influencing Work Ability am...,Factors Influencing Work Ability among the Wor...,The ability to work is an essential factor in ...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'An artificial neural network predic...,{'title': 'An artificial neural network predic...,{'title': 'An artificial neural network predic...,An artificial neural network prediction model ...,Exposure to indoor air pollutants can cause ad...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'Estimating occupational exposure to...,{'title': 'Estimating occupational exposure to...,{'title': 'Estimating occupational exposure to...,"Estimating occupational exposure to vocs, svoc...",Thailand is known for its agricultural product...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'Application of anthropometric data ...,{'title': 'Application of anthropometric data ...,{'title': 'Application of anthropometric data ...,Application of anthropometric data for elderly...,Home environments are the most common injury l...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,"{'title': 'Knowledge, attitudes, and practices...","{'title': 'Knowledge, attitudes, and practices...","{'title': 'Knowledge, attitudes, and practices...","Knowledge, attitudes, and practices (KAP) rela...",Tobacco is one of the crops using a lot of pes...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'An association between organophosph...,{'title': 'An association between organophosph...,{'title': 'An association between organophosph...,An association between organophosphate pestici...,BACKGROUND: Parkinson's disease (PD) is a ubiq...
0,56667622500,"Norkaew, Saowanee",9,NaN,NaN,NaN,{'title': 'Organophosphate pesticide exposure ...,{'title': 'Organophosphate pesticide exposure ...,{'title': 'Organophosphate pesticide exposure ...,Organophosphate pesticide exposure and dialkyl...,BACKGROUND: Chlorpyrifos and profenofos are or...
1,57222758601,"Chatchawan, Sapphasak",2,NaN,NaN,NaN,{'title': 'Can sentiment from news headlines e...,{'title': 'Can sentiment from news headlines e...,{'title': 'Can sentiment from news headlines e...,Can sentiment from news headlines explain stoc...,The objective of this study was to investigate...


In [29]:
df1.drop(['documents' , 'main_title' , 'main_abstract' , 'documents_number'], axis=1)

,author_id,author_name,citiations,h-index,co-authors,Title,Abstraction
0,56667622500,"Norkaew, Saowanee",NaN,NaN,NaN,Residential environment in relation to self-re...,Respiratory disease and its complication are t...
0,56667622500,"Norkaew, Saowanee",NaN,NaN,NaN,Anthropometric Measurement of Thai Older Farme...,Agricultural workers usually perform most occu...
0,56667622500,"Norkaew, Saowanee",NaN,NaN,NaN,Factors Influencing Work Ability among the Wor...,The ability to work is an essential factor in ...
0,56667622500,"Norkaew, Saowanee",NaN,NaN,NaN,An artificial neural network prediction model ...,Exposure to indoor air pollutants can cause ad...
0,56667622500,"Norkaew, Saowanee",NaN,NaN,NaN,"Estimating occupational exposure to vocs, svoc...",Thailand is known for its agricultural product...
0,56667622500,"Norkaew, Saowanee",NaN,NaN,NaN,Application of anthropometric data for elderly...,Home environments are the most common injury l...
0,56667622500,"Norkaew, Saowanee",NaN,NaN,NaN,"Knowledge, attitudes, and practices (KAP) rela...",Tobacco is one of the crops using a lot of pes...
0,56667622500,"Norkaew, Saowanee",NaN,NaN,NaN,An association between organophosphate pestici...,BACKGROUND: Parkinson's disease (PD) is a ubiq...
0,56667622500,"Norkaew, Saowanee",NaN,NaN,NaN,Organophosphate pesticide exposure and dialkyl...,BACKGROUND: Chlorpyrifos and profenofos are or...
1,57222758601,"Chatchawan, Sapphasak",NaN,NaN,NaN,Can sentiment from news headlines explain stoc...,The objective of this study was to investigate...


In [35]:
df1.to_excel('resercher_all.xls',index=False)

C:\Users\COM01\AppData\Local\Temp\ipykernel_3924\1064179885.py:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df1.to_excel('resercher_all.xls',index=False)
